数据加载

In [1]:
import pandas as pd

# demo 验证
train_df = pd.read_csv('Data/Movie_RS.csv')
print(train_df.shape)
train_df.head(1)

(10000, 13)


,ID,Movie_Name,Movie_Score,Review_Count,Movie_Star_Distribution,Collect_Date,Username,Post_Date,Score,User_Comment,User_Comment_Distribution,Comment_Like_Count,Movie_Tags
0,0,"1988年的妮可 Nico, 1988",7.5,565,15.2%48.2%32.3%3.4%0.8%,2019-10-05,尾黑,2018-06-23,3,成本低廉的PPT电影，用Nico生命中最后一年发生的事给Nico的歌配上情节，倒不算尴尬。女...,66%31%3%,4,"['音乐', '电影', '儿子', '丝绒', '人物', '传记', '传记片', '歌..."


#### 数据预处理

In [2]:
# 时间排序，将最近的排在前面
train_df = train_df.sort_values(by='Post_Date', ascending=False)

# 查看最后几行数据
train_df.tail()['Post_Date']

4507    2006-04-11
242     2006-02-03
3292    2005-12-18
2072    2005-10-13
204     2005-07-19
Name: Post_Date, dtype: object

In [3]:
# 去除空值
train_df.dropna(axis=0, how='any', inplace=True)

# 两列去除重复，保留最近的数据
train_df.drop_duplicates(
    subset=['Movie_Name', 'Username'], keep='first', inplace=True)

# 打印数据结构
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 2135 to 204
Data columns (total 13 columns):
ID                           9999 non-null int64
Movie_Name                   9999 non-null object
Movie_Score                  9999 non-null float64
Review_Count                 9999 non-null int64
Movie_Star_Distribution      9999 non-null object
Collect_Date                 9999 non-null object
Username                     9999 non-null object
Post_Date                    9999 non-null object
Score                        9999 non-null int64
User_Comment                 9999 non-null object
User_Comment_Distribution    9999 non-null object
Comment_Like_Count           9999 non-null int64
Movie_Tags                   9999 non-null object
dtypes: float64(1), int64(4), object(8)
memory usage: 1.1+ MB


增加时间过滤，不需要很久远的数据。

In [4]:
Time_Threshold = '2017-01-01'

# 直接对时间字段进行截断
train_df = train_df[train_df['Post_Date'] > Time_Threshold]

# 打印最后几行数据
train_df.tail()

,ID,Movie_Name,Movie_Score,Review_Count,Movie_Star_Distribution,Collect_Date,Username,Post_Date,Score,User_Comment,User_Comment_Distribution,Comment_Like_Count,Movie_Tags
8808,8917,不是冤家不聚头 不是冤家不聚頭,7.0,535,9.6%36.4%49.9%3.6%0.5%,2019-10-05,謎,2017-01-03,4,看过的第一个芳芳的高清，爱死芳芳了,45%52%3%,0,"['演技', '剧情', '电影', '芳姐', '温情', '喜剧', '家庭', '故事..."
3915,3977,一轮明月,7.9,4395,28.0%44.8%23.4%3.1%0.8%,2019-10-05,浪小二,2017-01-03,3,传记电影的通病，事件由一堆镜头PPT一样地飞入飞出，一带而过，你要自己查资料才能联起来。想说...,72%22%6%,0,"['电影', '大师', '人物', '感觉', '人生', '主旋律', '流水账', '..."
4024,4088,一轮明月,7.9,4395,28.0%44.8%23.4%3.1%0.8%,2019-10-05,Trady,2017-01-03,4,下午去了灵隐寺看到了弘一法师画像 朋友之前提到过这部电影 今天看正适合／对在杭州取景的那段特别喜欢,72%22%6%,0,"['电影', '大师', '人物', '感觉', '人生', '主旋律', '流水账', '..."
9797,9916,不要忘记哥哥 にぃにのことを忘れないで,7.4,2091,16.9%38.5%41.0%3.3%0.3%,2019-10-05,长泽小清新,2017-01-02,5,锦户亮的眼睛啊真的会说话,55%40%5%,0,"['演技', '锦户亮', '妈妈', '剧情', '眼泪', '片子', '绝症', '感..."
2639,2670,一年级生 The First Grader,8.1,2483,28.9%47.6%22.5%0.9%0.2%,2019-10-05,小鱼儿,2017-01-02,5,原来能自己阅读是如此幸福的事。我们的幸福都不是理所当然。,77%21%2%,0,"['老人', '故事', '电影', '孩子', '历史', '励志', '影片', '剧情..."


### 数据预处理

In [5]:
# 建立用户名和 id 映射的字典
user_dict = {value: index for index,
             value in enumerate(train_df['Username'].unique())}

# 开始对原数据进行转换
train_df['uid_int'] = train_df['Username'].apply(lambda x: user_dict[x])

# 用户 id 字典翻转
reverse_user_dict = {v: k for k, v in user_dict.items()}

# 建立电影名和 id 映射的字典
item_dict = {value: index for index, value in enumerate(
    train_df['Movie_Name'].unique())}

# 开始对原数据进行转换
train_df['item_int'] = train_df['Movie_Name'].apply(lambda x: item_dict[x])

# 电影 id 字典翻转
reverse_item_dict = {v: k for k, v in item_dict.items()}

#### 设置电影和用户特征

In [6]:
# 电影特征
items_f = ['Movie_Score', 'Review_Count', 'item_int', 'Movie_Name','Movie_Tags']

# 用户特征
users_f = ['uid_int','User_Comment_Distribution','Username']

#### 数据划分

用户交互表划分

In [7]:
user_post_event = train_df[['uid_int', 'item_int', 'Score', 'Post_Date']]
user_post_event.shape

(2435, 4)

In [8]:
# 找到每位用户的电影 id 序列
raw_movies = [user_post_event[user_post_event['uid_int'] == i]['item_int'].unique(
).tolist() for i in user_post_event['uid_int'].unique()]

# 对电影评论序列进行 str 处理
raw_id = []
for r_list in raw_movies:
    raw_id.append([str(i) for i in r_list])
    
len(raw_id)

2323

### 基于用户评论的最近邻电影推荐

In [9]:
from gensim import corpora, similarities
from gensim.models import Word2Vec
import multiprocessing

# 模型训练
%time model = Word2Vec(raw_id, window=3, size=300, workers=multiprocessing.cpu_count()*2, min_count=1)

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 29.2 ms


假设当前电影 id，并打印详细信息。

In [10]:
# 输入电影名字
movie_realname = '一轮明月'

# 电影名字转换 id
movie_id = str(item_dict[movie_realname])

# 打印电影信息
train_df.loc[train_df['item_int'].isin([int(movie_id)])].drop_duplicates(
    'item_int', keep='first', inplace=False)[items_f].head()

,Movie_Score,Review_Count,item_int,Movie_Name,Movie_Tags
3878,7.9,4395,3,一轮明月,"['电影', '大师', '人物', '感觉', '人生', '主旋律', '流水账', '..."


通过建立的电影 id 向量，找到最近的 tokn 个电影。

In [11]:
topn = 10

# 开始电影的近邻检索
%time ANN_List = [int(i[0]) for i in model.wv.most_similar(movie_id, topn=topn)]
ANN_List

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 715 µs


[10, 11, 17, 7, 15, 21, 1, 6, 12, 23]

打印近邻的电影信息。

In [12]:
# 数据去重，保留首先出现的数据。
ANN_List_Info = train_df.loc[train_df['item_int'].isin(
    ANN_List)].drop_duplicates('item_int', keep='first', inplace=False)


# 控制推荐质量，设置电影打分阈值 Threshold。
Threshold = 7.5
ANN_List_Info = ANN_List_Info[ANN_List_Info['Movie_Score'] > Threshold]

# 打印前五行数据
ANN_List_Info[items_f].head()

,Movie_Score,Review_Count,item_int,Movie_Name,Movie_Tags
4660,7.6,2620,1,七小福,"['电影', '故事', '时代', '师父', '传统', '感人', '片场', '京剧..."
9030,8.0,3581,6,不朽真情 Immortal Beloved,"['音乐', '电影', '爱情', '故事', '有点', '剧情', '天才', '狗血..."
6016,7.6,550,11,上帝的使者 الرسالة,"['宗教', '电影', '历史', '先知', '影片', '主角', '信仰', '镜头..."
5508,9.3,1010,12,三个臭皮匠 The Three Stooges,"['喜剧', '小时候', '时候', '经典', '电视', '电影', '电视剧', '..."
5107,7.7,3760,23,万夫莫敌 Invincible,"['励志', '电影', '橄榄球', '故事', '时候', '球队', '男人', '影..."


#### 应用推广
1. 用户电影之间的电影近邻推荐。
2. 用户用户之间的用户近邻推荐。
3. 将电影和用户的向量进行 inner dot 计算，形成用户个性化推荐和电影 push 推送。